# RAG System Debug & Analysis Notebook

This notebook provides comprehensive debugging and analysis tools for your SQL RAG system. 
Use this to understand:

- **What embeddings are being generated**
- **Which documents are retrieved and why**
- **What schema context is being injected**
- **What exactly is being sent to the LLM**
- **Why certain queries fail to find relevant tables**

## 1. Setup and Imports

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import time
from typing import List, Dict, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

# Add current directory to path
current_dir = Path.cwd()
if str(current_dir) not in sys.path:
    sys.path.append(str(current_dir))

# LangChain imports
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema.document import Document

# Local imports
from schema_manager import SchemaManager
from actions.llm_interaction import generate_answer_from_context, initialize_llm_client

# Visualization imports
try:
    from sklearn.manifold import TSNE
    from sklearn.decomposition import PCA
    from sklearn.metrics.pairwise import cosine_similarity
    SKLEARN_AVAILABLE = True
except ImportError:
    print("⚠️ sklearn not available - install with: pip install scikit-learn")
    SKLEARN_AVAILABLE = False

try:
    import umap
    UMAP_AVAILABLE = True
except ImportError:
    print("⚠️ umap not available - install with: pip install umap-learn")
    UMAP_AVAILABLE = False

print("✅ Setup complete")
print(f"📁 Current directory: {current_dir}")
print(f"🔬 Visualization tools: sklearn={SKLEARN_AVAILABLE}, umap={UMAP_AVAILABLE}")

## 2. Load and Inspect Vector Stores

In [ ]:
# Discover available vector stores
faiss_dir = current_dir / "faiss_indices"

if not faiss_dir.exists():
    print("❌ No faiss_indices directory found")
    print("Please run your embedding generation script first")
else:
    indices = [d for d in faiss_dir.iterdir() if d.is_dir() and d.name.startswith("index_")]
    
    print(f"📦 Found {len(indices)} vector stores:")
    for i, idx in enumerate(indices):
        index_file = idx / "index.faiss"
        pkl_file = idx / "index.pkl"
        
        if index_file.exists() and pkl_file.exists():
            size_mb = (index_file.stat().st_size + pkl_file.stat().st_size) / (1024*1024)
            print(f"  {i+1}. {idx.name} ({size_mb:.1f} MB)")
        else:
            print(f"  {i+1}. {idx.name} (INCOMPLETE - missing files)")
    
    # Store for later use
    available_indices = indices

## 3. Initialize Embedding Model and Load Vector Store

In [ ]:
# Initialize embedding model
print("🔄 Initializing embedding model...")
try:
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    
    # Test embedding generation
    test_embedding = embeddings.embed_query("test query")
    print(f"✅ Embedding model loaded: {len(test_embedding)}-dimensional vectors")
    
except Exception as e:
    print(f"❌ Failed to load embedding model: {e}")
    print("Make sure Ollama is running and nomic-embed-text model is available")
    embeddings = None

In [ ]:
# Select and load a vector store
if available_indices and embeddings:
    # Use the first available index (change this as needed)
    selected_index = available_indices[0]
    print(f"🔄 Loading vector store: {selected_index.name}")
    
    try:
        vector_store = FAISS.load_local(
            str(selected_index),
            embeddings,
            allow_dangerous_deserialization=True
        )
        
        # Get basic info about the vector store
        num_docs = len(vector_store.docstore._dict)
        print(f"✅ Vector store loaded: {num_docs} documents")
        
        # Sample some documents
        sample_docs = vector_store.similarity_search("sample query", k=3)
        print(f"📊 Sample documents loaded: {len(sample_docs)} examples")
        
    except Exception as e:
        print(f"❌ Failed to load vector store: {e}")
        vector_store = None
else:
    vector_store = None
    print("❌ No vector store available")

## 4. Schema Manager Setup

In [ ]:
# Look for schema files
schema_files = list(current_dir.glob("*.csv"))
schema_files.extend(list(current_dir.glob("*schema*.csv")))
schema_files.extend(list(current_dir.glob("../retail_system/**/*.csv")))

print(f"📋 Found {len(schema_files)} potential schema files:")
for f in schema_files[:5]:  # Show first 5
    print(f"  - {f.name} ({f.stat().st_size / 1024:.1f} KB)")

# Initialize schema manager if files found
schema_manager = None
if schema_files:
    # Try to initialize with the largest CSV file (likely the schema)
    largest_csv = max(schema_files, key=lambda f: f.stat().st_size)
    
    try:
        # Check if it has the right columns
        df_sample = pd.read_csv(largest_csv, nrows=5)
        required_cols = ['table_id', 'column', 'datatype']
        
        if all(col in df_sample.columns for col in required_cols):
            print(f"🔄 Initializing schema manager with: {largest_csv.name}")
            schema_manager = SchemaManager(str(largest_csv), verbose=True)
            print(f"✅ Schema manager loaded: {schema_manager.table_count} tables")
        else:
            print(f"⚠️ {largest_csv.name} doesn't have required schema columns")
            print(f"   Found columns: {list(df_sample.columns)}")
            print(f"   Required: {required_cols}")
    
    except Exception as e:
        print(f"❌ Failed to load schema manager: {e}")

if not schema_manager:
    print("⚠️ No schema manager available - schema injection testing will be limited")

## 5. Vector Embedding Analysis

In [ ]:
def analyze_query_embeddings(queries: List[str], embeddings_model, vector_store=None):
    """
    Analyze embeddings for a list of queries and show similarity patterns.
    """
    if not embeddings_model:
        print("❌ No embeddings model available")
        return
    
    print(f"🔄 Generating embeddings for {len(queries)} queries...")
    
    # Generate embeddings
    query_embeddings = []
    for i, query in enumerate(queries):
        try:
            embedding = embeddings_model.embed_query(query)
            query_embeddings.append(embedding)
            if i % 2 == 0:  # Progress indicator
                print(f"   {i+1}/{len(queries)} embeddings generated")
        except Exception as e:
            print(f"❌ Failed to embed query '{query[:50]}...': {e}")
            query_embeddings.append(None)
    
    # Filter out failed embeddings
    valid_embeddings = [(q, e) for q, e in zip(queries, query_embeddings) if e is not None]
    
    if not valid_embeddings:
        print("❌ No valid embeddings generated")
        return
    
    queries_valid = [q for q, e in valid_embeddings]
    embeddings_array = np.array([e for q, e in valid_embeddings])
    
    print(f"✅ Generated {len(embeddings_array)} embeddings with {embeddings_array.shape[1]} dimensions")
    
    # Calculate similarity matrix
    if SKLEARN_AVAILABLE:
        similarity_matrix = cosine_similarity(embeddings_array)
        
        # Plot similarity heatmap
        plt.figure(figsize=(10, 8))
        sns.heatmap(similarity_matrix, 
                   xticklabels=[q[:30] + '...' if len(q) > 30 else q for q in queries_valid],
                   yticklabels=[q[:30] + '...' if len(q) > 30 else q for q in queries_valid],
                   annot=True, fmt='.2f', cmap='coolwarm')
        plt.title('Query Embedding Similarity Matrix')
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()
    
    # If vector store available, test retrieval
    if vector_store:
        print("\n🔍 Testing retrieval for each query:")
        for query in queries_valid[:3]:  # Test first 3 queries
            print(f"\n📝 Query: {query[:60]}...")
            try:
                docs = vector_store.similarity_search(query, k=3)
                print(f"   📊 Retrieved {len(docs)} documents")
                
                for i, doc in enumerate(docs, 1):
                    content_preview = doc.page_content[:100].replace('\n', ' ')
                    source = doc.metadata.get('source', 'Unknown')
                    print(f"     {i}. {content_preview}... (Source: {source})")
                    
            except Exception as e:
                print(f"   ❌ Retrieval failed: {e}")
    
    return embeddings_array, queries_valid

# Test with sample queries
test_queries = [
    "How to join customer and order tables?",
    "Show me LEFT JOIN examples with multiple tables",
    "Customer analysis with aggregation functions",
    "Inventory management queries",
    "Calculate total sales by customer",
    "Find top customers by order count"
]

embedding_results = analyze_query_embeddings(test_queries, embeddings, vector_store)

## 6. Retrieval Pipeline Inspector

In [ ]:
def inspect_retrieval_pipeline(query: str, vector_store, k: int = 5, show_scores: bool = True):
    """
    Detailed inspection of the retrieval pipeline for a single query.
    """
    if not vector_store:
        print("❌ No vector store available")
        return None
    
    print(f"🔍 RETRIEVAL PIPELINE ANALYSIS")
    print(f"Query: '{query}'")
    print("=" * 70)
    
    try:
        # Step 1: Generate query embedding
        print("\n1️⃣ QUERY EMBEDDING GENERATION")
        start_time = time.time()
        query_embedding = embeddings.embed_query(query)
        embed_time = time.time() - start_time
        print(f"   ✅ Generated {len(query_embedding)}-dimensional embedding in {embed_time:.3f}s")
        print(f"   📊 Embedding stats: min={min(query_embedding):.3f}, max={max(query_embedding):.3f}, mean={np.mean(query_embedding):.3f}")
        
        # Step 2: Similarity search
        print("\n2️⃣ VECTOR SIMILARITY SEARCH")
        start_time = time.time()
        
        if show_scores:
            # Use similarity_search_with_score for detailed analysis
            results_with_scores = vector_store.similarity_search_with_score(query, k=k)
            search_time = time.time() - start_time
            
            print(f"   ✅ Retrieved {len(results_with_scores)} documents in {search_time:.3f}s")
            
            # Analyze results
            docs = []
            for i, (doc, score) in enumerate(results_with_scores, 1):
                docs.append(doc)
                print(f"\n   📄 Result {i} (Similarity Score: {score:.4f})")
                print(f"      Source: {doc.metadata.get('source', 'Unknown')}")
                print(f"      Content Preview: {doc.page_content[:150].replace(chr(10), ' ')}...")
                
                # Show metadata
                if doc.metadata:
                    relevant_metadata = {k: v for k, v in doc.metadata.items() 
                                       if k in ['description', 'table', 'tables', 'joins'] and v}
                    if relevant_metadata:
                        print(f"      Metadata: {relevant_metadata}")
        else:
            docs = vector_store.similarity_search(query, k=k)
            search_time = time.time() - start_time
            print(f"   ✅ Retrieved {len(docs)} documents in {search_time:.3f}s")
        
        # Step 3: Content analysis
        print("\n3️⃣ CONTENT ANALYSIS")
        total_content_length = sum(len(doc.page_content) for doc in docs)
        print(f"   📊 Total content length: {total_content_length:,} characters")
        
        # Analyze query term overlap
        query_terms = set(query.lower().split())
        print(f"   🔍 Query terms: {query_terms}")
        
        for i, doc in enumerate(docs, 1):
            content_terms = set(doc.page_content.lower().split())
            overlap = query_terms.intersection(content_terms)
            overlap_ratio = len(overlap) / len(query_terms) if query_terms else 0
            print(f"   📄 Doc {i}: {len(overlap)} matching terms ({overlap_ratio:.1%}) - {overlap}")
        
        return {
            'query': query,
            'query_embedding': query_embedding,
            'retrieved_docs': docs,
            'embed_time': embed_time,
            'search_time': search_time,
            'total_content_length': total_content_length
        }
        
    except Exception as e:
        print(f"❌ Retrieval pipeline error: {e}")
        import traceback
        traceback.print_exc()
        return None

# Test the retrieval pipeline with different queries
test_query = "How to calculate inventory turnover using JOIN between inventory and sales tables?"
retrieval_results = inspect_retrieval_pipeline(test_query, vector_store, k=4)

## 7. Schema Injection Analysis

In [ ]:
def analyze_schema_injection(query: str, retrieved_docs: List[Document], schema_manager=None):
    """
    Analyze how schema injection works for a given query and retrieved documents.
    """
    print(f"📋 SCHEMA INJECTION ANALYSIS")
    print(f"Query: '{query}'")
    print("=" * 70)
    
    if not schema_manager:
        print("⚠️ No schema manager available - cannot test schema injection")
        return None
    
    try:
        # Step 1: Extract tables from retrieved documents
        print("\n1️⃣ TABLE EXTRACTION FROM RETRIEVED DOCUMENTS")
        extracted_tables = schema_manager.extract_tables_from_documents(retrieved_docs)
        print(f"   🔍 Extracted {len(extracted_tables)} table names: {extracted_tables}")
        
        # Step 2: Analyze each document for table references
        print("\n2️⃣ DOCUMENT-LEVEL TABLE ANALYSIS")
        for i, doc in enumerate(retrieved_docs, 1):
            doc_tables = schema_manager.extract_tables_from_content(doc.page_content)
            print(f"   📄 Document {i}: {doc_tables}")
            print(f"      Content sample: {doc.page_content[:100].replace(chr(10), ' ')}...")
            
            # Check metadata for table information
            if hasattr(doc, 'metadata') and doc.metadata:
                table_metadata = {k: v for k, v in doc.metadata.items() 
                                if k in ['table', 'tables', 'joins'] and v}
                if table_metadata:
                    print(f"      Table metadata: {table_metadata}")
        
        # Step 3: Schema lookup for extracted tables
        print("\n3️⃣ SCHEMA LOOKUP RESULTS")
        schema_found = []
        schema_missing = []
        
        for table in extracted_tables:
            schema = schema_manager.get_schema_for_table(table)
            if schema:
                schema_found.append((table, len(schema)))
                print(f"   ✅ {table}: {len(schema)} columns found")
            else:
                schema_missing.append(table)
                print(f"   ❌ {table}: No schema found")
        
        # Step 4: Generate complete schema context
        print("\n4️⃣ GENERATED SCHEMA CONTEXT")
        if extracted_tables:
            schema_context = schema_manager.get_relevant_schema(extracted_tables)
            print(f"   📊 Generated schema context ({len(schema_context)} characters):")
            print("   " + "-" * 50)
            print(schema_context[:500] + "..." if len(schema_context) > 500 else schema_context)
            print("   " + "-" * 50)
        else:
            schema_context = ""
            print("   ⚠️ No schema context generated - no tables extracted")
        
        # Step 5: Test with manual table names if extraction failed
        if not extracted_tables:
            print("\n5️⃣ MANUAL TABLE EXTRACTION TEST")
            # Try common table names from the query
            query_lower = query.lower()
            common_tables = ['customers', 'orders', 'inventory', 'products', 'sales', 'users']
            manual_tables = [t for t in common_tables if t in query_lower]
            
            if manual_tables:
                print(f"   🧪 Testing manual table names: {manual_tables}")
                manual_schema = schema_manager.get_relevant_schema(manual_tables)
                if manual_schema:
                    print(f"   ✅ Found schema for manual tables ({len(manual_schema)} chars)")
                    print(manual_schema[:200] + "...")
                else:
                    print("   ❌ No schema found for manual table names either")
        
        return {
            'extracted_tables': extracted_tables,
            'schema_found': schema_found,
            'schema_missing': schema_missing,
            'schema_context': schema_context
        }
        
    except Exception as e:
        print(f"❌ Schema injection analysis failed: {e}")
        import traceback
        traceback.print_exc()
        return None

# Test schema injection analysis
if retrieval_results:
    schema_analysis = analyze_schema_injection(
        retrieval_results['query'], 
        retrieval_results['retrieved_docs'], 
        schema_manager
    )

## 8. LLM Context Inspector

In [ ]:
def inspect_llm_context(query: str, retrieved_docs: List[Document], schema_context: str = ""):
    """
    Show exactly what context is being sent to the LLM.
    """
    print(f"🤖 LLM CONTEXT INSPECTOR")
    print(f"Query: '{query}'")
    print("=" * 70)
    
    # Step 1: Build document context
    print("\n1️⃣ DOCUMENT CONTEXT CONSTRUCTION")
    context_parts = []
    total_doc_chars = 0
    
    for i, doc in enumerate(retrieved_docs, 1):
        content = f"Document {i}:\n{doc.page_content}"
        
        # Add metadata if available
        metadata = doc.metadata
        if metadata.get('description'):
            content += f"\nDescription: {metadata['description']}"
        if metadata.get('table'):
            content += f"\nTables: {metadata['table']}"
        if metadata.get('joins'):
            content += f"\nJoins: {metadata['joins']}"
        
        context_parts.append(content)
        total_doc_chars += len(content)
        print(f"   📄 Document {i}: {len(content)} characters")
    
    doc_context = "\n\n" + "="*50 + "\n\n".join(context_parts)
    print(f"   📊 Total document context: {len(doc_context)} characters")
    
    # Step 2: Add schema context
    print("\n2️⃣ SCHEMA CONTEXT ADDITION")
    if schema_context:
        full_context = f"{schema_context}\n\n{doc_context}"
        print(f"   📋 Schema context: {len(schema_context)} characters")
        print(f"   📊 Total context with schema: {len(full_context)} characters")
    else:
        full_context = doc_context
        print(f"   ⚠️ No schema context available")
        print(f"   📊 Total context: {len(full_context)} characters")
    
    # Step 3: Build complete prompt
    print("\n3️⃣ COMPLETE PROMPT CONSTRUCTION")
    system_prompt = (
        "You are an expert SQL analyst helping answer questions about a retail analytics codebase. "
        "Use ONLY the provided context to answer the user's question. If the answer is not contained "
        "within the context, respond with 'I don't know based on the provided context.'"
    )
    
    complete_prompt = f"{system_prompt}\n\nContext:\n{full_context}\n\nUser question: {query}\n\nAnswer:"
    
    print(f"   📝 System prompt: {len(system_prompt)} characters")
    print(f"   🎯 Complete prompt: {len(complete_prompt)} characters")
    
    # Estimate token count (rough approximation)
    estimated_tokens = len(complete_prompt.split()) * 1.3
    print(f"   🔢 Estimated tokens: ~{int(estimated_tokens)}")
    
    # Step 4: Show prompt sections
    print("\n4️⃣ PROMPT PREVIEW")
    print("   📋 System Prompt:")
    print(f"      {system_prompt[:100]}...")
    
    if schema_context:
        print("   📊 Schema Context (first 200 chars):")
        print(f"      {schema_context[:200].replace(chr(10), ' ')}...")
    
    print("   📄 Document Context (first 200 chars):")
    print(f"      {doc_context[:200].replace(chr(10), ' ')}...")
    
    print("   ❓ User Query:")
    print(f"      {query}")
    
    return {
        'system_prompt': system_prompt,
        'document_context': doc_context,
        'schema_context': schema_context,
        'complete_prompt': complete_prompt,
        'estimated_tokens': int(estimated_tokens),
        'context_breakdown': {
            'system_prompt_chars': len(system_prompt),
            'document_context_chars': len(doc_context),
            'schema_context_chars': len(schema_context),
            'total_chars': len(complete_prompt)
        }
    }

# Test LLM context inspection
if retrieval_results and schema_analysis:
    llm_context_analysis = inspect_llm_context(
        retrieval_results['query'],
        retrieval_results['retrieved_docs'],
        schema_analysis.get('schema_context', '')
    )

## 9. End-to-End Pipeline Test

In [ ]:
def run_complete_rag_pipeline(query: str, vector_store, schema_manager=None, k: int = 4):
    """
    Run the complete RAG pipeline with full visibility.
    """
    print(f"🚀 COMPLETE RAG PIPELINE TEST")
    print(f"Query: '{query}'")
    print("=" * 80)
    
    pipeline_results = {
        'query': query,
        'timestamps': {},
        'errors': []
    }
    
    try:
        # Step 1: Retrieve documents
        print("\n📚 STEP 1: DOCUMENT RETRIEVAL")
        start_time = time.time()
        
        retrieved_docs = vector_store.similarity_search(query, k=k)
        pipeline_results['timestamps']['retrieval'] = time.time() - start_time
        pipeline_results['retrieved_docs'] = retrieved_docs
        
        print(f"   ✅ Retrieved {len(retrieved_docs)} documents in {pipeline_results['timestamps']['retrieval']:.3f}s")
        
        # Step 2: Extract tables and get schema
        print("\n📋 STEP 2: SCHEMA EXTRACTION")
        start_time = time.time()
        
        schema_context = ""
        extracted_tables = []
        
        if schema_manager:
            extracted_tables = schema_manager.extract_tables_from_documents(retrieved_docs)
            schema_context = schema_manager.get_relevant_schema(extracted_tables)
            
            pipeline_results['timestamps']['schema'] = time.time() - start_time
            pipeline_results['extracted_tables'] = extracted_tables
            pipeline_results['schema_context'] = schema_context
            
            print(f"   ✅ Extracted {len(extracted_tables)} tables: {extracted_tables}")
            print(f"   📊 Generated schema context: {len(schema_context)} characters")
            print(f"   ⏱️ Schema processing time: {pipeline_results['timestamps']['schema']:.3f}s")
        else:
            print("   ⚠️ No schema manager - skipping schema injection")
            pipeline_results['timestamps']['schema'] = 0
        
        # Step 3: Build context
        print("\n🔧 STEP 3: CONTEXT CONSTRUCTION")
        start_time = time.time()
        
        # Build document context
        context_parts = []
        for i, doc in enumerate(retrieved_docs, 1):
            content = f"Document {i}:\n{doc.page_content}"
            
            # Add metadata
            metadata = doc.metadata
            if metadata.get('description'):
                content += f"\nDescription: {metadata['description']}"
            if metadata.get('table'):
                content += f"\nTables: {metadata['table']}"
            
            context_parts.append(content)
        
        doc_context = "\n\n" + "="*50 + "\n\n".join(context_parts)
        
        # Combine with schema
        if schema_context:
            full_context = f"{schema_context}\n\n{doc_context}"
        else:
            full_context = doc_context
        
        pipeline_results['timestamps']['context'] = time.time() - start_time
        pipeline_results['full_context'] = full_context
        
        print(f"   📊 Context length: {len(full_context)} characters")
        print(f"   ⏱️ Context construction time: {pipeline_results['timestamps']['context']:.3f}s")
        
        # Step 4: Generate answer with LLM
        print("\n🤖 STEP 4: LLM GENERATION")
        start_time = time.time()
        
        try:
            # Try to generate answer using the LLM interaction module
            answer, token_usage = generate_answer_from_context(
                query=query,
                context=full_context,
                retries=2
            )
            
            pipeline_results['timestamps']['llm'] = time.time() - start_time
            pipeline_results['answer'] = answer
            pipeline_results['token_usage'] = token_usage
            
            print(f"   ✅ Generated answer in {pipeline_results['timestamps']['llm']:.3f}s")
            print(f"   🔢 Token usage: {token_usage}")
            print(f"   📝 Answer length: {len(answer)} characters")
            
        except Exception as e:
            pipeline_results['errors'].append(f"LLM generation failed: {e}")
            print(f"   ❌ LLM generation failed: {e}")
            pipeline_results['answer'] = None
            pipeline_results['token_usage'] = {}
        
        # Step 5: Results summary
        print("\n📊 PIPELINE SUMMARY")
        total_time = sum(pipeline_results['timestamps'].values())
        print(f"   ⏱️ Total pipeline time: {total_time:.3f}s")
        print(f"      - Retrieval: {pipeline_results['timestamps']['retrieval']:.3f}s")
        print(f"      - Schema: {pipeline_results['timestamps']['schema']:.3f}s")
        print(f"      - Context: {pipeline_results['timestamps']['context']:.3f}s")
        print(f"      - LLM: {pipeline_results['timestamps'].get('llm', 0):.3f}s")
        
        if pipeline_results['answer']:
            print("\n🎯 GENERATED ANSWER:")
            print("-" * 50)
            print(pipeline_results['answer'])
            print("-" * 50)
        
        if pipeline_results['errors']:
            print("\n⚠️ ERRORS ENCOUNTERED:")
            for error in pipeline_results['errors']:
                print(f"   - {error}")
        
        return pipeline_results
        
    except Exception as e:
        pipeline_results['errors'].append(f"Pipeline error: {e}")
        print(f"❌ Pipeline failed: {e}")
        import traceback
        traceback.print_exc()
        return pipeline_results

# Test the complete pipeline
test_pipeline_query = "Show me how to calculate customer lifetime value using SQL joins"
complete_pipeline_results = run_complete_rag_pipeline(
    test_pipeline_query, 
    vector_store, 
    schema_manager, 
    k=3
)

## 10. Interactive Query Testing Widget

In [ ]:
# Install ipywidgets if not available
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    WIDGETS_AVAILABLE = True
except ImportError:
    print("⚠️ ipywidgets not available - install with: pip install ipywidgets")
    WIDGETS_AVAILABLE = False

if WIDGETS_AVAILABLE:
    def create_interactive_tester():
        """
        Create an interactive widget for testing queries.
        """
        # Create widgets
        query_input = widgets.Textarea(
            value="How to join customer and order tables?",
            placeholder="Enter your test query here...",
            description="Query:",
            layout=widgets.Layout(width='80%', height='80px')
        )
        
        k_slider = widgets.IntSlider(
            value=3,
            min=1,
            max=10,
            description="Retrieve (k):",
            style={'description_width': 'initial'}
        )
        
        test_button = widgets.Button(
            description="🧪 Test Query",
            button_style='primary',
            layout=widgets.Layout(width='200px')
        )
        
        output_area = widgets.Output()
        
        def test_query_interactive(b):
            with output_area:
                clear_output(wait=True)
                
                query = query_input.value.strip()
                k = k_slider.value
                
                if not query:
                    print("❌ Please enter a query")
                    return
                
                if not vector_store:
                    print("❌ No vector store available")
                    return
                
                print(f"🧪 Testing query: '{query}' (k={k})")
                print("=" * 60)
                
                try:
                    # Quick retrieval test
                    start = time.time()
                    docs = vector_store.similarity_search(query, k=k)
                    retrieval_time = time.time() - start
                    
                    print(f"📊 Retrieved {len(docs)} documents in {retrieval_time:.3f}s\n")
                    
                    # Show results
                    for i, doc in enumerate(docs, 1):
                        print(f"📄 Result {i}:")
                        print(f"   Content: {doc.page_content[:120]}...")
                        print(f"   Source: {doc.metadata.get('source', 'Unknown')}")
                        print()
                    
                    # Schema extraction if available
                    if schema_manager:
                        tables = schema_manager.extract_tables_from_documents(docs)
                        if tables:
                            print(f"🏷️ Extracted tables: {tables}")
                            schema = schema_manager.get_relevant_schema(tables[:3])  # Limit for display
                            if schema:
                                print(f"📋 Schema context: {len(schema)} characters")
                        else:
                            print("⚠️ No tables extracted from retrieved documents")
                    
                except Exception as e:
                    print(f"❌ Test failed: {e}")
        
        test_button.on_click(test_query_interactive)
        
        # Layout
        controls = widgets.VBox([
            widgets.HTML("<h3>🔍 Interactive Query Tester</h3>"),
            query_input,
            widgets.HBox([k_slider, test_button]),
            output_area
        ])
        
        return controls
    
    # Create and display the interactive tester
    if vector_store:
        interactive_tester = create_interactive_tester()
        display(interactive_tester)
    else:
        print("❌ Cannot create interactive tester - no vector store available")
        
else:
    print("📝 Manual testing section - modify the cell below to test different queries:")

## 11. Performance Analysis & Recommendations

In [ ]:
def analyze_system_performance():
    """
    Analyze the performance characteristics of your RAG system.
    """
    print("📈 RAG SYSTEM PERFORMANCE ANALYSIS")
    print("=" * 60)
    
    analysis = {
        'vector_store_info': {},
        'embedding_performance': {},
        'retrieval_performance': {},
        'recommendations': []
    }
    
    # Vector store analysis
    if vector_store:
        print("\n🗄️ VECTOR STORE ANALYSIS")
        num_docs = len(vector_store.docstore._dict)
        analysis['vector_store_info'] = {
            'total_documents': num_docs,
            'embedding_dimension': len(embeddings.embed_query("test")) if embeddings else "Unknown"
        }
        
        print(f"   📊 Total documents: {num_docs:,}")
        if embeddings:
            print(f"   📏 Embedding dimension: {analysis['vector_store_info']['embedding_dimension']}")
        
        # Sample document analysis
        sample_docs = list(vector_store.docstore._dict.values())[:5]
        if sample_docs:
            doc_lengths = [len(doc.page_content) for doc in sample_docs]
            avg_length = sum(doc_lengths) / len(doc_lengths)
            print(f"   📄 Average document length: {avg_length:.0f} characters")
            analysis['vector_store_info']['avg_doc_length'] = avg_length
    
    # Embedding performance test
    if embeddings:
        print("\n⚡ EMBEDDING PERFORMANCE TEST")
        test_queries = [
            "short query",
            "This is a medium length query with several words to test embedding performance",
            "This is a much longer query that contains multiple sentences and various SQL-related terms like SELECT, JOIN, WHERE, GROUP BY, and ORDER BY to simulate real user queries that might be submitted to the RAG system for processing and analysis."
        ]
        
        embedding_times = []
        for i, query in enumerate(test_queries):
            try:
                start = time.time()
                embeddings.embed_query(query)
                embed_time = time.time() - start
                embedding_times.append(embed_time)
                print(f"   Query {i+1} ({len(query)} chars): {embed_time:.3f}s")
            except Exception as e:
                print(f"   Query {i+1}: Failed - {e}")
        
        if embedding_times:
            avg_embedding_time = sum(embedding_times) / len(embedding_times)
            analysis['embedding_performance'] = {
                'avg_time': avg_embedding_time,
                'min_time': min(embedding_times),
                'max_time': max(embedding_times)
            }
            print(f"   📊 Average embedding time: {avg_embedding_time:.3f}s")
    
    # Retrieval performance test
    if vector_store:
        print("\n🔍 RETRIEVAL PERFORMANCE TEST")
        test_k_values = [1, 3, 5, 10]
        test_query = "customer order analysis with joins"
        
        retrieval_times = {}
        for k in test_k_values:
            try:
                start = time.time()
                docs = vector_store.similarity_search(test_query, k=k)
                retrieval_time = time.time() - start
                retrieval_times[k] = retrieval_time
                print(f"   k={k}: {retrieval_time:.3f}s ({len(docs)} docs)")
            except Exception as e:
                print(f"   k={k}: Failed - {e}")
        
        analysis['retrieval_performance'] = retrieval_times
    
    # Schema manager performance
    if schema_manager:
        print("\n📋 SCHEMA MANAGER ANALYSIS")
        stats = schema_manager.get_schema_stats()
        print(f"   📊 Total tables: {stats['total_tables']:,}")
        print(f"   📊 Total columns: {stats['total_columns']:,}")
        print(f"   📊 Average columns per table: {stats['avg_columns_per_table']:.1f}")
        
        # Test table extraction performance
        test_content = "SELECT * FROM customers c JOIN orders o ON c.customer_id = o.customer_id JOIN products p ON o.product_id = p.product_id"
        start = time.time()
        tables = schema_manager.extract_tables_from_content(test_content)
        extraction_time = time.time() - start
        print(f"   🔍 Table extraction time: {extraction_time:.4f}s ({len(tables)} tables found)")
    
    # Generate recommendations
    print("\n💡 PERFORMANCE RECOMMENDATIONS")
    recommendations = []
    
    if vector_store and analysis['vector_store_info'].get('total_documents', 0) > 10000:
        recommendations.append("Consider implementing hierarchical indexing for large document collections")
    
    if analysis.get('embedding_performance', {}).get('avg_time', 0) > 1.0:
        recommendations.append("Embedding generation is slow - consider using a faster embedding model or caching")
    
    if analysis.get('retrieval_performance'):
        retrieval_times = analysis['retrieval_performance']
        if retrieval_times.get(5, 0) > 0.5:
            recommendations.append("Retrieval is slow - consider optimizing FAISS index or reducing k values")
    
    if not schema_manager:
        recommendations.append("No schema manager detected - consider implementing smart schema injection")
    
    if not recommendations:
        recommendations.append("System performance looks good! Consider monitoring in production.")
    
    analysis['recommendations'] = recommendations
    
    for i, rec in enumerate(recommendations, 1):
        print(f"   {i}. {rec}")
    
    return analysis

# Run performance analysis
performance_analysis = analyze_system_performance()

## 12. Export Debug Results

In [ ]:
def export_debug_results():
    """
    Export all debug results to JSON for further analysis.
    """
    print("💾 EXPORTING DEBUG RESULTS")
    print("=" * 50)
    
    # Collect all results
    debug_export = {
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
        'system_info': {
            'vector_store_available': vector_store is not None,
            'schema_manager_available': schema_manager is not None,
            'embeddings_available': embeddings is not None,
            'sklearn_available': SKLEARN_AVAILABLE,
            'umap_available': UMAP_AVAILABLE,
            'widgets_available': WIDGETS_AVAILABLE
        },
        'retrieval_results': retrieval_results if 'retrieval_results' in locals() else None,
        'schema_analysis': schema_analysis if 'schema_analysis' in locals() else None,
        'llm_context_analysis': llm_context_analysis if 'llm_context_analysis' in locals() else None,
        'complete_pipeline_results': complete_pipeline_results if 'complete_pipeline_results' in locals() else None,
        'performance_analysis': performance_analysis if 'performance_analysis' in locals() else None
    }
    
    # Clean up non-serializable objects
    def clean_for_json(obj):
        if isinstance(obj, dict):
            return {k: clean_for_json(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [clean_for_json(item) for item in obj]
        elif hasattr(obj, 'page_content'):  # Document object
            return {
                'page_content': str(obj.page_content)[:500] + '...' if len(str(obj.page_content)) > 500 else str(obj.page_content),
                'metadata': dict(obj.metadata) if hasattr(obj, 'metadata') else {}
            }
        elif isinstance(obj, np.ndarray):
            return f"numpy_array_shape_{obj.shape}"
        elif callable(obj):
            return f"callable_{obj.__class__.__name__}"
        else:
            try:
                json.dumps(obj)  # Test if serializable
                return obj
            except (TypeError, ValueError):
                return str(obj)
    
    cleaned_export = clean_for_json(debug_export)
    
    # Save to file
    export_filename = f"rag_debug_results_{int(time.time())}.json"
    export_path = current_dir / export_filename
    
    try:
        with open(export_path, 'w') as f:
            json.dump(cleaned_export, f, indent=2)
        
        file_size = export_path.stat().st_size / 1024  # KB
        print(f"✅ Debug results exported to: {export_filename}")
        print(f"📊 File size: {file_size:.1f} KB")
        
        # Show summary
        print("\n📋 Export Summary:")
        for key, value in debug_export.items():
            if key == 'system_info':
                available_components = sum(1 for v in value.values() if v)
                print(f"   {key}: {available_components}/{len(value)} components available")
            elif value is not None:
                print(f"   {key}: ✅ Included")
            else:
                print(f"   {key}: ❌ Not available")
        
        return export_path
        
    except Exception as e:
        print(f"❌ Failed to export results: {e}")
        return None

# Export results
export_path = export_debug_results()

## Summary & Next Steps

This notebook provides comprehensive debugging capabilities for your RAG system. Here's what you can use it for:

### 🔍 **Debugging Capabilities**
1. **Vector Embedding Analysis** - See what embeddings are generated and their similarity patterns
2. **Retrieval Pipeline Inspection** - Understand exactly which documents are retrieved and why
3. **Schema Injection Analysis** - Debug why certain tables aren't being found or extracted
4. **LLM Context Inspector** - See the exact prompt being sent to your language model
5. **End-to-End Testing** - Run complete pipeline with full visibility
6. **Performance Analysis** - Identify bottlenecks and optimization opportunities

### 🚀 **Next Steps**
1. **Run different test queries** to understand retrieval patterns
2. **Experiment with k values** to optimize retrieval count
3. **Test schema injection** with queries containing table names
4. **Analyze failed queries** to understand system limitations
5. **Use exported results** for deeper analysis and system tuning

### 💡 **Common Issues & Solutions**
- **No tables extracted**: Check if your documents contain proper SQL syntax or table metadata
- **Poor retrieval**: Consider adjusting embedding model or similarity thresholds
- **Slow performance**: Use the performance analysis to identify bottlenecks
- **LLM failures**: Check context length and ensure proper prompt formatting

**This notebook is your debugging companion - modify and extend it as needed for your specific use cases!**